In [1]:
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from time import sleep
import random
import requests
from selenium.webdriver.common.by import By
import pandas as pd
import threading
from queue import Queue
from IPython.display import HTML
import capsolver

In [2]:
def path_to_image_html(path):
    return f'<img src="{path}">'

def load_data_page(driver, s_page, n_page):
    driver.maximize_window()
    web_url = f"https://www.lazada.vn/truyen-thong-am-nhac-sach/?page={s_page}"
    driver.get(web_url)
    sleep(60)
    
#     capsolver.api_key="CAP-4ACD04C0EA3A0B8B2AA19C551C3F64D6"
#     solution = capsolver.solve({
#             "type":"ReCaptchaV2TaskProxyLess",
#             "websiteKey":"6LeKU1AnAAAAANDF_Hx4z4OgLH1CJRwTO4IQ0Rfa",
#             "websiteURL": web_url,
#         })
#     token = solution('gRecaptchaResponse')
#     driver.execute_script(f"document.getElementById('g-recaptcha-response').value = '{token}'"
                          
    dfs = []
    for i_page in range(n_page):
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        for i in range(10):
            driver.execute_script(f"window.scrollBy(0, {scroll_height/10});")
            sleep(3)
        elems = driver.find_elements(By.CSS_SELECTOR , ".RfADt a")
        title = [elem.text for elem in elems]
        links = [elem.get_attribute('href') for elem in elems]

        elems_price = driver.find_elements(By.CSS_SELECTOR , ".aBrP0")
        price = [elem_price.text for elem_price in elems_price]

        elems_discount = driver.find_elements(By.CSS_SELECTOR , ".WNoq3")
        discount = [elem.text for elem in elems_discount]

        img_urls =[]
        i_img=0
        while(True):
            try:
                elem_image = driver.find_elements(By.CSS_SELECTOR, ".picture-wrapper.jBwCF img")[i_img]
                img_url = elem_image.get_attribute('src')
                img_urls.append(img_url)
        #         response = requests.get(img_url)
        #         if response.status_code == 200:
        #             img_path = f'D:/Clawl data/Image_book_lazada/{title[i]}.png'
        #             with open(img_path, 'wb') as f:
        #                 f.write(response.content)
        #             sleep(random.randint(1,3))
                i_img+=1
            except Exception as e:
                if str(e) == "list index out of range":
                    break
                print(f"Errol for element {i_img} with {str(e)}")

        df = pd.DataFrame(list(zip(title, price, links, discount, img_urls)), 
                      columns = ['title','price','link','discount','image_url'])
        df['image'] = df['image_url'].apply(path_to_image_html)
        df['page'] = s_page + i_page
        dfs.append(df)
        
        driver.execute_script(f"window.scrollTo(0, {scroll_height*0.9});")
        driver.find_element(By.CSS_SELECTOR, 'button.ant-pagination-item-link span[aria-label="right"]').click()

    total_df = pd.concat(dfs, ignore_index=True)
    return total_df
    
def crawl_data_multi_browsers(n_thread,n_page):
    drivers, threads, result = [], [], []
    q = Queue()
    for i in range(n_thread):
        chromedriver_path = 'D:/Phần mềm/chromedriver-win64/chromedriver.exe'
        driver = webdriver.Chrome(service=Service(chromedriver_path))
        drivers.append(driver)
        
    for i_thread, driver in enumerate(drivers):
        s_page = i_thread * n_page + 1
        t = threading.Thread(target=lambda q, driver, s_page, n_page: q.put(load_data_page(driver, s_page, n_page)), 
                             args = (q, driver, s_page, n_page))
        threads.append(t)
        t.start()
        
    for t in threads:
        t.join()
        
    while not q.empty():
        result.append(q.get())
        
    return result

In [3]:
n_thread=2
n_page=5
result=crawl_data_multi_browsers(n_thread,n_page) 
results = pd.concat(result, ignore_index=True)
result_html =  HTML(results.to_html(escape=False))
result_html

,title,price,link,discount,image_url,image,page
0,Sách - Lớp Học Mật Ngữ - Tuyển Tập Đặc Biệt Vol. 6 - Phương Nam Book,60.000 ₫,https://www.lazada.vn/products/sach-lop-hoc-mat-ngu-tuyen-tap-dac-biet-vol-6-phuong-nam-book-i2661881527.html,Coin save 5.400 ₫,https://img.lazcdn.com/g/p/cb301500af50ccbf742bb597f6fa4902.jpg_400x400q75.jpg_.webp,,6
1,"Takenote từ vựng Tiếng Trung HSK1- HSK6 (5456 từ, có đủ từ loại, nghĩa, pinyin, ô luyện viết, ví dụ đi kèm)-Nhà sách Ôn Luyện",98.000 ₫,https://www.lazada.vn/products/takenote-tu-vung-tieng-trung-hsk1-hsk6-5456-tu-co-du-tu-loai-nghia-pinyin-o-luyen-viet-vi-du-di-kem-nha-sach-on-luyen-i2419566596.html,39% Off\nCoin save 1.960 ₫,https://img.lazcdn.com/g/p/850b8f74dee15c4b052698a31691f437.jpg_400x400q75.jpg_.webp,,6
2,Sách - Lưỡng Giới (Giải Pulitzer 2003) (Bìa Mềm) - Phương Nam Book,230.300 ₫,https://www.lazada.vn/products/sach-luong-gioi-giai-pulitzer-2003-bia-mem-phuong-nam-book-i1026284657.html,30% Off\nCoin save 23.030 ₫,https://img.lazcdn.com/g/p/6bac0fb12fb045ce6fa07ad7bf302594.jpg_400x400q75.jpg_.webp,,6
3,Sách - Cherry Magic! - Tập 10 - Phương Nam Book,109.000 ₫,https://www.lazada.vn/products/sach-cherry-magic-tap-10-phuong-nam-book-i2661644059.html,Coin save 9.810 ₫,https://img.lazcdn.com/g/p/eaca9f0689d13a46063bd8499911890f.jpg_400x400q75.jpg_.webp,,6
4,Sách - Thiền Sư Và Em bé 5 tuổi - Tác giả: Thích Nhất Hạnh,84.000 ₫,https://www.lazada.vn/products/sach-thien-su-va-em-be-5-tuoi-tac-gia-thich-nhat-hanh-i923124354.html,30% Off,https://img.lazcdn.com/g/p/b062d171d5fca26b8f050cc80c6821ac.jpg_400x400q75.jpg_.webp,,6
5,Sách - 500 trò chơi rèn luyện IQ (4-10 tuổi) - Khả năng tập trung (sách bản quyền),56.250 ₫,https://www.lazada.vn/products/sach-500-tro-choi-ren-luyen-iq-4-10-tuoi-kha-nang-tap-trung-sach-ban-quyen-i1963382652.html,25% Off\nCoin save 563 ₫,https://img.lazcdn.com/g/p/367b839f0df9c3af53159aa564bdf6f5.png_400x400q75.png_.webp,,6
6,Đạo Phật Vỡ Lòng,52.440 ₫,https://www.lazada.vn/products/dao-phat-vo-long-i622426381.html,24% Off,https://img.lazcdn.com/g/p/f429ba1d24e7acb5cc20ed756719f096.jpg_400x400q75.jpg_.webp,,6
7,Sách - Bảy Bước Tới Mùa Hè (Tái bản năm 2022) (Khổ nhỏ) - Phương Nam Book,52.000 ₫,https://www.lazada.vn/products/sach-bay-buoc-toi-mua-he-tai-ban-nam-2022-kho-nho-phuong-nam-book-i2661401511.html,Coin save 4.680 ₫,https://img.lazcdn.com/g/p/bfb4591fbe505be562396c0b54f7c696.jpg_400x400q75.jpg_.webp,,6
8,Sách - Nếu em không phải một giấc mơ,71.200 ₫,https://www.lazada.vn/products/sach-neu-em-khong-phai-mot-giac-mo-i1497683721.html,20% Off,https://img.lazcdn.com/g/p/3723cfa3916d497c0c4008feef50766b.jpg_400x400q75.jpg_.webp,,6
9,Topik II Hướng dẫn thi Topik II theo cấp bậc - Topik II 합격 레시피,198.000 ₫,https://www.lazada.vn/products/topik-ii-huong-dan-thi-topik-ii-theo-cap-bac-topik-ii-i1738228898.html,,https://img.lazcdn.com/g/ff/kf/S4947a800ae364fb598285959da5332b5A.jpg_400x400q75.jpg_.webp,,6


In [4]:
results.to_csv('D:/Clawl data/book_lazada.csv', index=False)